# Introduction to Reinforcement Learning
Reinforcement Learning can be thought of as a third paradigm of statistical learning, separated from Supervised (e.g., Regression models) and unsupervised learning(e.g. clustering).

One of the main characteristics of Reinforcement Learning is that it consists of a goal-directed agent, which interacts with an environment. The agent can observe the state of the environment, and perform actions that change the state of the environment. The agent also receives rewards from the environment, which are used to evaluate the agent's actions. The goal of the agent is to maximize the total reward it receives from the environment. The environment is uncertain. leading the agent to try to balance exploitation (leveraging previous successful experiences) *vs* exploration(trying out new strategies), in its decisions.

Whenever the agent take an action, the environment is modified as a result. Therefore, the correct choice of action must take into account indirect delayed consequences of all previous actions. Thus the environment can be thought of as a Markov Decision Process (MDP).

## Getting Started: the K-armed Bandit Problem
The K-armed bandit problem is a simple example of a Reinforcement Learning problem. The agent is faced repeatedly with a choice. It can choose between K different actions, and receives a reward based on the action it chooses, the reward comes from a fixed probability distribution over the actions. The goal of the agent is to maximize the total reward it receives over time. The action chosen at time $t$ is denoted as $A_t$. The reward received is denoted as $R_t$. The expected value of an action $a$ is denoted as $q_*(a)$:

$$q_*(a) = \mathrm{E}[R_t \mid A_t = a].$$

The estimated value of an action $a$ at time $t$ is denoted as $Q_t(a)$. The value of an action is the expected reward given that action is selected. The value of an action is not known to the agent, and must be estimated based on the rewards received. The agent must balance exploration (trying out different actions to estimate their value) and exploitation (choosing the action with the highest estimated value).
